In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [152]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from urllib import request
from urllib.request import urlopen
from string import ascii_lowercase as alc

In [153]:
base_url = "http://www.ufcstats.com/statistics/fighters?char="

In [154]:
html = urlopen(base_url + 'a')
soup = BeautifulSoup(html, 'lxml')
table = soup.find('table')

In [155]:
#grabbing the headers for the dataset / cleaning them
header = [th.get_text(strip=True) for th in table.find_all('th')]

In [156]:
#putting header in a data frame and then transposing it
df = pd.DataFrame(header)
df = df.T
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt


In [143]:
#grabbing stats for every page
for char in alc:
    url = base_url + char + "&page=all"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.find_all('tr')
        list_rows = []
        for row in rows:
            cells = row.find_all('td')
            str_cells = [str(cell.get_text(strip=True)) for cell in cells]
            list_rows.append(str_cells)
            
            #checking for belt
            has_belt = 1 if any(cell.find('img', class_='b-list__icon') for cell in cells) else 0
            str_cells.append(has_belt)
            
            list_rows.append(str_cells)
            
        df_page = pd.DataFrame(list_rows)
        all_dataframes.append(df_page)
        
        print(f"Page for char '{char}': {soup.title.text}")
    else:
        print(f"Failed to retrieve page for char '{char}' with status code {response.status_code}")
        
final_df = pd.concat(all_dataframes, ignore_index=True)

Page for char 'a': 
    Stats | UFC
  
Page for char 'b': 
    Stats | UFC
  
Page for char 'c': 
    Stats | UFC
  
Page for char 'd': 
    Stats | UFC
  
Page for char 'e': 
    Stats | UFC
  
Page for char 'f': 
    Stats | UFC
  
Page for char 'g': 
    Stats | UFC
  
Page for char 'h': 
    Stats | UFC
  
Page for char 'i': 
    Stats | UFC
  
Page for char 'j': 
    Stats | UFC
  
Page for char 'k': 
    Stats | UFC
  
Page for char 'l': 
    Stats | UFC
  
Page for char 'm': 
    Stats | UFC
  
Page for char 'n': 
    Stats | UFC
  
Page for char 'o': 
    Stats | UFC
  
Page for char 'p': 
    Stats | UFC
  
Page for char 'q': 
    Stats | UFC
  
Page for char 'r': 
    Stats | UFC
  
Page for char 's': 
    Stats | UFC
  
Page for char 't': 
    Stats | UFC
  
Page for char 'u': 
    Stats | UFC
  
Page for char 'v': 
    Stats | UFC
  
Page for char 'w': 
    Stats | UFC
  
Page for char 'x': 
    Stats | UFC
  
Page for char 'y': 
    Stats | UFC
  
Page for char 'z': 
    S

In [144]:
frames = [df, final_df]
df2 = pd.concat(frames)
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,NaN
0,0,None,None,None,None,None,None,None,None,None,None,NaN
1,0,None,None,None,None,None,None,None,None,None,None,NaN
2,,0,None,None,None,None,None,None,None,None,None,NaN
3,,0,None,None,None,None,None,None,None,None,None,NaN


In [145]:
df3 = df2.rename(columns=df2.iloc[0])
start_index = 3
selected_rows = df3[start_index:]
selected_rows.head()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt,NaN
2,,0,None,None,None,None,None,None,None,None,None,NaN
3,,0,None,None,None,None,None,None,None,None,None,NaN
4,Tom,Aaron,,--,155 lbs.,--,,5,3,0,,0.0
5,Tom,Aaron,,--,155 lbs.,--,,5,3,0,,0.0
6,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0,,0.0


In [146]:
selected_rows.to_csv('UFC_Stats.csv', index = False)